In [ ]:
import pandas as pd
from glob import glob
import libarchive # `pip install libarchive-c` and using with libarchive 3.4.x
from io import BytesIO

In [ ]:
total_size = 0
header = True
for fn in sorted(glob('secc/secc*.7z')):
    print(fn)
    with libarchive.file_reader(fn) as e:
        for entry in e:
            total_size += entry.size
            print(total_size / (1024*1024))
            with BytesIO() as f:
                for i, block in enumerate(entry.get_blocks()):
                    f.write(block)
                f.seek(0)
                #df = pd.read_csv(f, nrows=10)
                df = pd.read_csv(f, usecols=['state', 'head_of_hh', 'gender', 'age', 'social_cat'])
        
    df.dropna(subset=['head_of_hh'], inplace=True)
    df.state = df.state.str.lower()
    df.gender = df.gender.str.lower()
    df.social_cat = df.social_cat.str.lower()
    df['name'] = df.head_of_hh.str.split('\\').str[0]
    df.name = df.name.str.lower()
    del df['head_of_hh']
    df.to_csv('/opt/data/secc/secc_state_name_age_sex_social_cat.csv', mode='a', header=header, index=False)
    header = False